In [0]:
import pyspark
from pyspark import SparkConf,SparkContext
conf=SparkConf().setAppName("CarDetails")
sc=SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Spark Dataframes").getOrCreate()

In [0]:
rdd=sc.textFile("/FileStore/tables/cars.tsv")
headers=rdd.first()

In [0]:
print(headers)

make	Model	MPG	Cylinders	Engine Disp	Horsepower	Weight	Accelerate	Year	Origin


In [0]:
rdd=rdd.filter(lambda x: x!=headers).map(lambda x: x.split('\t'))

In [0]:
columns=headers.split('\t')
print(columns)

['make', 'Model', 'MPG', 'Cylinders', 'Engine Disp', 'Horsepower', 'Weight', 'Accelerate', 'Year', 'Origin']


In [0]:
dfrdd=rdd.toDF(columns)

In [0]:
dfrdd.show()

+---------+--------------------+---+---------+-----------+----------+------+----------+----+--------+
|     make|               Model|MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+---------+--------------------+---+---------+-----------+----------+------+----------+----+--------+
|      amc|  amc ambassador dpl| 15|        8|        390|       190|  3850|       8.5|  70|American|
|      amc|         amc gremlin| 21|        6|        199|        90|  2648|        15|  70|American|
|      amc|          amc hornet| 18|        6|        199|        97|  2774|      15.5|  70|American|
|      amc|       amc rebel sst| 16|        8|        304|       150|  3433|        12|  70|American|
|    buick|buick estate wago...| 14|        8|        455|       225|  3086|        10|  70|American|
|    buick|   buick skylark 320| 15|        8|        350|       165|  3693|      11.5|  70|American|
|chevrolet|chevrolet chevell...| 18|        8|        307|       130|  3504|      

In [0]:
rdd1=rdd.map(lambda x:[x[0],x[1],int(x[2]),int(x[3]), int(x[4]), int(x[5]) , int(x[6]), float(x[7]), int(x[8]), x[9]])

In [0]:
rdd1.collect()

Out[56]: [['amc', 'amc ambassador dpl', 15, 8, 390, 190, 3850, 8.5, 70, 'American'],
 ['amc', 'amc gremlin', 21, 6, 199, 90, 2648, 15.0, 70, 'American'],
 ['amc', 'amc hornet', 18, 6, 199, 97, 2774, 15.5, 70, 'American'],
 ['amc', 'amc rebel sst', 16, 8, 304, 150, 3433, 12.0, 70, 'American'],
 ['buick',
  'buick estate wagon (sw)',
  14,
  8,
  455,
  225,
  3086,
  10.0,
  70,
  'American'],
 ['buick', 'buick skylark 320', 15, 8, 350, 165, 3693, 11.5, 70, 'American'],
 ['chevrolet',
  'chevrolet chevelle malibu',
  18,
  8,
  307,
  130,
  3504,
  12.0,
  70,
  'American'],
 ['chevrolet', 'chevrolet impala', 14, 8, 454, 220, 4354, 9.0, 70, 'American'],
 ['chevrolet',
  'chevrolet monte carlo',
  15,
  8,
  400,
  150,
  3761,
  9.5,
  70,
  'American'],
 ['chevy', 'chevy c20', 10, 8, 307, 200, 4376, 15.0, 70, 'American'],
 ['dodge', 'dodge challenger se', 15, 8, 383, 170, 3563, 10.0, 70, 'American'],
 ['dodge', 'dodge d200', 11, 8, 318, 210, 4382, 13.5, 70, 'American'],
 ['ford', 'for

In [0]:
from pyspark.sql.types import StructType, StructField , StringType , IntegerType , FloatType

In [0]:
test=StructType([
    StructField("make",StringType(),True),
    StructField("Model",StringType(),True),
    StructField("MPG",IntegerType(),True),
    StructField("Cylinders",IntegerType(),True),
    StructField("Engine Disp",IntegerType(),True),
    StructField("Horsepower",IntegerType(),True),
    StructField("Weight",IntegerType(),True),
    StructField("Accelerate",FloatType(),True),
    StructField("Year",IntegerType(),True),
    StructField("Origin",StringType(),True)
    
])

In [0]:
dfrdd1=spark.createDataFrame(rdd1, schema=test)

In [0]:
dfrdd1.show()

+---------+--------------------+---+---------+-----------+----------+------+----------+----+--------+
|     make|               Model|MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+---------+--------------------+---+---------+-----------+----------+------+----------+----+--------+
|      amc|  amc ambassador dpl| 15|        8|        390|       190|  3850|       8.5|  70|American|
|      amc|         amc gremlin| 21|        6|        199|        90|  2648|      15.0|  70|American|
|      amc|          amc hornet| 18|        6|        199|        97|  2774|      15.5|  70|American|
|      amc|       amc rebel sst| 16|        8|        304|       150|  3433|      12.0|  70|American|
|    buick|buick estate wago...| 14|        8|        455|       225|  3086|      10.0|  70|American|
|    buick|   buick skylark 320| 15|        8|        350|       165|  3693|      11.5|  70|American|
|chevrolet|chevrolet chevell...| 18|        8|        307|       130|  3504|      

In [0]:
dfrdd1.printSchema()

root
 |-- make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- MPG: integer (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: integer (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: float (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



In [0]:
from pyspark.sql.functions import sum,count,avg, min , max, round
dfrdd2=dfrdd1.filter(dfrdd1.Origin=='American').groupBy('make').agg(round(avg('weight')))

In [0]:
dfrdd2.show()

+---------+---------------------+
|     make|round(avg(weight), 0)|
+---------+---------------------+
|    buick|               3760.0|
|  pontiac|               4328.0|
|      amc|               3239.0|
| plymouth|               3383.0|
|    chevy|               4376.0|
|    dodge|               4300.0|
|     ford|               3792.0|
|chevrolet|               3461.0|
|       hi|               4732.0|
|  mercury|               2220.0|
+---------+---------------------+



In [0]:
rdd2=dfrdd2.rdd.map(tuple)

In [0]:
rdd2.collect()

Out[71]: [('buick', 3760.0),
 ('pontiac', 4328.0),
 ('amc', 3239.0),
 ('plymouth', 3383.0),
 ('chevy', 4376.0),
 ('dodge', 4300.0),
 ('ford', 3792.0),
 ('chevrolet', 3461.0),
 ('hi', 4732.0),
 ('mercury', 2220.0)]